In [4]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", None)

In [23]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, countDistinct
spark = SparkSession.builder.getOrCreate()

In [6]:
ls /home/gnana/football-lakehouse/silver

players_silver/


In [14]:
games = spark.read.parquet("/home/gnana/football-lakehouse/bronze/games")

In [15]:
games.count()

74026

In [16]:
games.printSchema()

root
 |-- game_id: integer (nullable = true)
 |-- competition_id: string (nullable = true)
 |-- season: integer (nullable = true)
 |-- round: string (nullable = true)
 |-- date: date (nullable = true)
 |-- home_club_id: integer (nullable = true)
 |-- away_club_id: integer (nullable = true)
 |-- home_club_goals: integer (nullable = true)
 |-- away_club_goals: integer (nullable = true)
 |-- home_club_position: integer (nullable = true)
 |-- away_club_position: integer (nullable = true)
 |-- home_club_manager_name: string (nullable = true)
 |-- away_club_manager_name: string (nullable = true)
 |-- stadium: string (nullable = true)
 |-- attendance: integer (nullable = true)
 |-- referee: string (nullable = true)
 |-- url: string (nullable = true)
 |-- home_club_formation: string (nullable = true)
 |-- away_club_formation: string (nullable = true)
 |-- home_club_name: string (nullable = true)
 |-- away_club_name: string (nullable = true)
 |-- aggregate: timestamp (nullable = true)
 |-- comp

In [17]:
games.limit(5).toPandas()

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,away_club_position,home_club_manager_name,away_club_manager_name,stadium,attendance,referee,url,home_club_formation,away_club_formation,home_club_name,away_club_name,aggregate,competition_type
0,2818490,FAC,2016,4th round deciders,2017-02-08,1003,22,3,1,NaN,NaN,Claudio Ranieri,Steve McClaren,King Power Stadium,31648,Mike Jones,https://www.transfermarkt.co.uk/spielbericht/index/spielbericht/2818490,4-4-1-1,4-1-4-1,Leicester City Football Club,None,2026-02-08 03:01:00,domestic_cup
1,2873429,UKRP,2017,First Round,2017-07-09,18105,13017,2,0,NaN,NaN,Andriy Chikh,Sergiy Yashchenko,Stadion SKIF,400,Vadym Zaprutnyak,https://www.transfermarkt.co.uk/spielbericht/index/spielbericht/2873429,4-4-2 double 6,4-2-3-1,PFK Lviv,None,2026-02-08 02:00:00,domestic_cup
2,2874417,DKP,2017,First Round,2017-08-08,1894,444,1,0,NaN,NaN,Per Frandsen,Henrik Gundersen,PRO VENTILATION Arena,1126,Anders Poulsen,https://www.transfermarkt.co.uk/spielbericht/index/spielbericht/2874417,4-4-2,4-4-2,Hvidovre IF,None,2026-02-08 01:00:00,domestic_cup
3,2917419,CL,2017,Group H,2017-11-01,16,2784,1,1,NaN,NaN,Peter Bosz,Georgios Donis,SIGNAL IDUNA PARK,64500,Matej Jug,https://www.transfermarkt.co.uk/spielbericht/index/spielbericht/2917419,4-3-3 Attacking,5-3-2,Borussia Dortmund,None,2026-02-08 01:01:00,international_cup
4,2928256,NLP,2017,Second Round,2017-10-24,317,3892,3,0,NaN,NaN,Marino Pusic,Wilfred van Leeuwen,De Grolsch Veste,17100,Edwin van de Graaf,https://www.transfermarkt.co.uk/spielbericht/index/spielbericht/2928256,4-3-3 Defending,4-3-3 Attacking,Football Club Twente,None,2026-02-08 03:00:00,domestic_cup


In [18]:
games_silver = games.select(
    col("game_id"),
    col("season"),
    col("date"),
    col("home_club_id"),
    col("away_club_id"),
    col("home_club_goals"),
    col("away_club_goals"),
    col("stadium"),
    col("attendance"),
    col("competition_id")
).dropDuplicates(["game_id"])

In [19]:
games_silver.limit(5).toPandas()

,game_id,season,date,home_club_id,away_club_id,home_club_goals,away_club_goals,stadium,attendance,competition_id
0,2219795,2012,2012-08-31,631,13,1,4,Stade Louis-II,14312,USC
1,2221748,2012,2012-08-19,158,44,2,1,EWR-Arena Worms,6263,DFB
2,2221749,2012,2012-08-18,115,16,0,3,Wohninvest-Weserstadion,19325,DFB
3,2221753,2012,2012-08-18,2652,79,0,5,Karl-Liebknecht-Stadion,7200,DFB
4,2221759,2012,2012-08-20,109,27,0,4,Jahnstadion,12500,DFB


In [20]:
games_silver.count()

74026

In [25]:
games_silver.select(countDistinct("game_id")).toPandas()

,count(DISTINCT game_id)
0,74026


In [26]:
games_silver.write.mode("overwrite").parquet("/home/gnana/football-lakehouse/silver/games_silver")